# CLASSIFICATION

In [46]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
import random
import os

from sklearn.metrics import accuracy_score

In [47]:
def seed_everything(seed=42):
    """"Seed everything.
    """   
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [48]:
seed_everything()

In [10]:
train_set = pd.read_csv("data/train_set.txt",sep = ' ')
test_set = pd.read_csv("data/test_set.txt",sep = ' ')

In [11]:
X_train = pd.DataFrame(train_set).copy()
del X_train['rain']
del X_train['rain_class']
del X_train['rain_log']

X_test = pd.DataFrame(test_set).copy()
del X_test['rain']
del X_test['rain_class']
del X_test['rain_log']

In [23]:
train_set['rain_class'] = train_set['rain_class'].astype("category")
test_set['rain_class'] = test_set['rain_class'].astype("category")

In [25]:
Y_train = train_set['rain']
Y_test = test_set['rain']

In [26]:
Y_train_class = train_set['rain_class']
Y_test_class = test_set['rain_class']


In [101]:
X_train.head()

,ff,t,td,hu,dd,precip_log,ws_arome,p3031_arome,u10_arome,v10_arome,...,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
1,-0.086213,1.164777,0.910535,-0.798235,0.520054,-0.820399,-0.036595,0.529785,0.801953,0.400758,...,0,0,0,1,0,0,0,0,0,0
2,-0.215296,-1.471529,-1.661333,-0.454718,0.695888,0.078895,-0.257664,1.323143,0.556898,-0.888709,...,1,0,0,0,0,0,0,0,0,0
4,-1.468680,0.129100,-0.187510,-1.056853,0.129012,-0.820399,-1.317523,-0.827820,-0.605044,-0.693706,...,0,0,1,0,0,0,0,0,0,0
5,1.028401,-1.238485,-1.421240,-0.491033,1.022205,0.269433,0.464623,1.063498,1.186435,-0.717044,...,0,0,0,0,0,0,0,0,0,0
6,-1.465538,-1.273800,-1.288810,0.075769,-0.850480,-0.612447,-1.277939,-0.588190,-0.432199,0.107395,...,0,0,0,0,0,0,0,0,0,1


## 1. K nearest neighbors

The completeness parameter `k` is optimised on a predefined grid by minimising the estimated error by cross-validation; scikit-learn offers many cross-validation options. 

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
# Optimisation of k
param_grid = [{"n_neighbors": list(range(1, 20))}]
knn = GridSearchCV(KNeighborsClassifier(weights = "distance"), scoring = "accuracy", param_grid = param_grid, cv=10, n_jobs=-1, refit = True)
knnOpt = knn.fit(X_train, Y_train_class)  
# optimal parameter
# knnOpt.best_params_["n_neighbors"]
print("Best Mean cross-validated accuracy = %f, Best parameter = %s" % (knnOpt.best_score_, knnOpt.best_params_))

Best Mean cross-validated accuracy = 0.540000, Best parameter = {'n_neighbors': 18}


The prediction accuracy in the test set

In [36]:
# Estimation of the prediction accuracy on the test sample
print("Prediction accuracy in the test sample : ", knnOpt.score(X_test, Y_test_class))


Prediction accuracy in the test sample :  0.5072463768115942


In [54]:
# Prediction of the test sample
y_hat_class = knnOpt.predict(X_test)
print("Accuracy score =", accuracy_score(y_true = Y_test_class, y_pred = y_hat_class))

# confusion matrix
table = pd.crosstab(y_hat_class, Y_test_class)
print("Confusion matrix")
print(table)

Accuracy score = 0.5072463768115942
Confusion matrix
rain_class  high_rain  low_rain  no_rain
row_0                                   
high_rain          20        11       11
low_rain           12        37       23
no_rain             1        10       13


## 2. Decision tree

In [90]:
from sklearn.tree import DecisionTreeClassifier
# Optimisation of shaft depth
param_grid = [{"max_depth": range(2,10), "min_samples_split" : range(2,10), "min_samples_leaf": range(1,5) }]
tree = GridSearchCV(DecisionTreeClassifier(max_features = 'auto', min_impurity_decrease = 1e-3, random_state = 42), scoring = "accuracy", param_grid = param_grid, cv=10, n_jobs=-1, refit = True)
treeOpt = tree.fit(X_train, Y_train_class)
# Optimal parameter
print("Best Mean cross-validated accuracy = %f, Best parameter = %s" % (treeOpt.best_score_, treeOpt.best_params_))

Best Mean cross-validated accuracy = 0.512727, Best parameter = {'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 2}


In [91]:
# Estimation of the prediction error on the test sample
treeOpt.score(X_test, Y_test_class)

0.4855072463768116

In [92]:
# Prediction of the test sample
y_hat_class = treeOpt.predict(X_test)
print("Accuracy score =", accuracy_score(y_true = Y_test_class, y_pred = y_hat_class))
# confusion matrix
table = pd.crosstab(y_hat_class, Y_test_class)
print(table)

Accuracy score = 0.4855072463768116
rain_class  high_rain  low_rain  no_rain
row_0                                   
high_rain          14         6        4
low_rain           17        39       29
no_rain             2        13       14


    The `low_rain` class seems to be difficult to predict!

## 3. Random Forest

In [96]:
from sklearn.ensemble import RandomForestClassifier 
# Parameters' definitions
forest = RandomForestClassifier(n_estimators = 500, 
   criterion='gini', max_depth=None,
   min_samples_split=2, min_samples_leaf=1, 
   max_features='auto', max_leaf_nodes=None,
   bootstrap=True, oob_score=True)
# Training
rfFit = forest.fit(X_train,Y_train_class)
# Out-of-bag error on the train sample
print(1-rfFit.oob_score_)
# Out-of-bag error on the test sample
print(1-rfFit.score(X_test,Y_test_class))

0.46545454545454545
0.5072463768115942


Hyper parameters tunning by cross validation

In [99]:
param=[{"n_estimators" : range(200, 500, 100) , "max_features": range(2,5), "max_depth" : range(10, 20, 2) }]
rf = GridSearchCV(RandomForestClassifier(random_state=42, criterion="entropy"),
        param, cv = 5, n_jobs=-1)
rfOpt = rf.fit(X_train, Y_train_class)
print("Best Mean cross-validated accuracy = %f, Best parameter = %s" % (rfOpt.best_score_, rfOpt.best_params_))

Best Mean cross-validated accuracy = 0.545455, Best parameter = {'max_depth': 12, 'max_features': 4, 'n_estimators': 300}


In [100]:
# Prediction of the test sample
y_pred_test = treeOpt.predict(X_test)
print("Accuracy score =", accuracy_score(y_true = Y_test_class, y_pred = y_pred_test))
# confusion matrix
table = pd.crosstab(y_pred_test, Y_test_class)
print(table)

Accuracy score = 0.4855072463768116
rain_class  high_rain  low_rain  no_rain
row_0                                   
high_rain          14         6        4
low_rain           17        39       29
no_rain             2        13       14


### Gradient Boosting applying to Random Forest

In [107]:
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

In [105]:
from xgboost import XGBRFClassifier

In [108]:
# define the model
model = XGBRFClassifier(n_estimators = 500, subsample = 0.99, random_state = 42, metric = "accuracy", )

In [ ]:
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))